In [40]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings,OpenAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

In [3]:
load_dotenv(dotenv_path=find_dotenv(filename="D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
examples=[
    {"input":"2+2","output":"4"},
    {"input":"2+3","output":"5"}
    ]

In [13]:
examplePrompts=ChatPromptTemplate.from_messages(
messages=[
    ("human","{input}"),
    ("ai","{output}")
    ]
)

In [14]:
fewShotPrompt=FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=examplePrompts
)

In [17]:
print(fewShotPrompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [18]:
finalPrompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a Wondrous Wizard of Maths"),
        fewShotPrompt,
        ("human","{input}")
    ]
)

In [29]:
chain=RunnablePassthrough()|finalPrompt|llmGemini|StrOutputParser()|RunnableLambda(lambda s:s.strip())

In [31]:
chain.invoke(input={"input":"6*67"})

'402'

<h3>Using Example Selectors</h3>

In [36]:
examples=[
    {"input":"2+2","output":"4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {"input": "Write me a poem about the moon", "output": "One for the moon, and one for me, who are we to talk about the moon?"},
]

In [37]:
examplePrompt=ChatPromptTemplate.from_messages(
    messages= [
        ("human","{input}"),
        ("ai","{output}")
    ]
)

In [55]:
exampleSelector=SemanticSimilarityExampleSelector.from_examples(
    vectorstore_cls=FAISS,
    examples=examples,
    k=2,
    embeddings=SentenceTransformerEmbeddings()
)

C:\Users\MSI 1\AppData\Local\Temp\ipykernel_10904\2595075800.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings()


In [67]:
fewShotPrompt=FewShotChatMessagePromptTemplate(
    example_selector=exampleSelector,
    example_prompt=examplePrompt,
    input_variables=["enter"]  # Human Input not the input from the example prompt
)

In [63]:
finalPrompt=ChatPromptTemplate.from_messages(messages=[
    ("human","You are a Maths Wizard and also a Poet"),
    fewShotPrompt,
    ("human","{enter}")
])

In [64]:
finalPrompt.invoke(input={"enter":"6*67"})

ChatPromptValue(messages=[HumanMessage(content='You are a Maths Wizard and also a Poet', additional_kwargs={}, response_metadata={}), HumanMessage(content='2+4', additional_kwargs={}, response_metadata={}), AIMessage(content='6', additional_kwargs={}, response_metadata={}), HumanMessage(content='2+3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='6*67', additional_kwargs={}, response_metadata={})])

In [65]:
chain=RunnablePassthrough()|finalPrompt|llmGemini|StrOutputParser()|RunnableLambda(lambda s:s.strip())

In [66]:
chain.invoke(input={"enter":"6*67"})

'402'